In [1]:
import kfp
import kfp.components as comp
import kfp.compiler as compiler
import kfp.dsl as dsl

# Init an experiment

In [2]:
EXPERIMENT_NAME = 'SLEQ_BASICS'
PIPELINE_FILENAME = 'simple_flipcoin.tar.gz'
PIPELINE_RUN_NAME = '[basic] simple flipcoin'

In [3]:
client = kfp.Client()

existing_experiments = client.list_experiments().experiments

if existing_experiments is not None:
    experiment = next(iter([exp for exp in existing_experiments if exp.name == EXPERIMENT_NAME]), None)
else:
    experiment = None
    
if experiment is None:
    experiment = client.create_experiment(EXPERIMENT_NAME)
    print('Experiment %s created with ID %s' % (experiment.name, experiment.id))
else:
    print('Experiment already exists with id %s' % experiment.id)

Experiment already exists with id 57935e68-839a-492e-8140-98fcb83a10d0


# Defines simple pipeline

In [4]:
def add(a: float,  b: float) -> float:
    print('adding %s and %s' % (a, b))
    return a + b

add_operation = comp.func_to_container_op(add)

In [5]:
def mul(a: float,  b: float) -> float:
    print('multiplying %s and %s' % (a, b))
    return a * b

mul_operation = comp.func_to_container_op(mul)

In [11]:
def simple_print(content: str):
    print(content)

print_op = comp.func_to_container_op(simple_print)

In [9]:
class RandomFlipCoin(dsl.ContainerOp):
    def __init__(self):
        super(RandomFlipCoin, self).__init__(
            name='Random boolean',
            image='python:alpine3.6',
            command=['sh', '-c'],
            arguments=['python -c "import random; result = \'add\' if random.randint(0,1) == 0 '
                 'else \'mul\'; print(result)" | tee /tmp/output'],
            file_outputs={ 'output': '/tmp/output' }
        )

In [15]:
@dsl.pipeline(
    name='simple_random',
    description='my first pipeline with condition'
)
def pipeline(a: float, b: float) -> float:
    flip = RandomFlipCoin()
    with dsl.Condition(flip.output == 'add'):
        iner_op = add_operation(a, b)
        print_op(iner_op.output)
    with dsl.Condition(flip.output == 'mul'):
        iner_op = mul_operation(a, b)
        print_op(iner_op.output)

pipeline_func = compiler.Compiler().compile(pipeline, PIPELINE_FILENAME)

# Running the pipeline

In [16]:
arguments = { 'a': 2, 'b': 10 }

client.run_pipeline(experiment.id, PIPELINE_RUN_NAME, PIPELINE_FILENAME, arguments)

{'created_at': datetime.datetime(2019, 3, 11, 18, 36, 45, tzinfo=tzlocal()),
 'description': None,
 'error': None,
 'id': '9f56bdcc-442c-11e9-8c13-080027606df2',
 'metrics': None,
 'name': '[basic] simple flipcoin',
 'pipeline_spec': {'parameters': [{'name': 'a', 'value': '2'},
                                  {'name': 'b', 'value': '10'}],
                   'pipeline_id': None,
                   'pipeline_manifest': None,
                   'workflow_manifest': '{"apiVersion": '
                                        '"argoproj.io/v1alpha1", "kind": '
                                        '"Workflow", "metadata": '
                                        '{"generateName": "simple-random-"}, '
                                        '"spec": {"arguments": {"parameters": '
                                        '[{"name": "a"}, {"name": "b"}]}, '
                                        '"entrypoint": "simple-random", '
                                        '"serviceAccountName"